In [1]:
# !pip install yfinance


In [2]:
# dependencies
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine, Column, Integer, String, Date, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from datetime import date
import sqlite3
from datetime import datetime



In [3]:
# Creates a connection to our DB. if file doesn't exist, we create it
# echo=True is verbose
engine = create_engine("sqlite:///pstocks.sqlite", echo=True)
conn = engine.connect()
session = Session(engine)
Base = declarative_base()

# # Define our pet table
# probably deprecated after i found out about df.to_sql method
# class stock(Base):
#     __tablename__ = 'stock'
#     id = Column(Integer, primary_key=True)
#     name = Column(String)
#     type = Column(String)
#     date = Column(Date)
# # creates table
# Base.metadata.create_all(engine)

In [4]:
# choose tickers here
# tickerStrings = ['AAPL', 'MSFT','TSLA','META','TTCF']

In [5]:
#import csv file with the list of S&P 500 companies
splist = pd.read_csv("sp500_constituents.csv")
# print df
splist

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care
...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


In [6]:
#s&p500 tickers
sp500 = splist.rename(columns={'Symbol':'Ticker'})
# print df
sp500

,Ticker,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care
...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


In [ ]:
# variable for the list for the for loop
tickerStrings = sp500['Ticker']

# yfinance code to separate tickers into pretty dataframe
df_list = list()
for ticker in tickerStrings:
    data = yf.download(ticker, group_by="Ticker", period='2y')
    data['Ticker'] = ticker  # add this column because the dataframe doesn't contain a column with the ticker
    
    df_list.append(data)

# combine all dataframes into a single dataframe
tickerDf = pd.concat(df_list)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- INFO: No data found, symbol may be delisted
[*********************100%***********

In [ ]:
# reset index to prepare for merge
tickerindex = tickerDf.reset_index()

tickerindex

In [ ]:
# merge the sp500 df to get add the ticker name
mergeticker = tickerindex.merge(sp500, how = 'right', on = 'Ticker')

mergeticker

In [ ]:
#find percent change from day to day
pct_stock = mergeticker['Close'].pct_change()*100

#add percent change to df
mergeticker['ClosePerChange'] = pct_stock

# dropping Nan rows
dropnan = mergeticker.dropna()
dropnan

In [ ]:
# dropping columns
closeticker = dropnan.drop(columns =['High','Low','Open','Adj Close'])

# set index to date
cleanedticker = closeticker.set_index('Date')

cleanedticker

In [ ]:
# cleanedticker["__id"] = 1
cleanedticker.insert(0, '__id', range(0, len(cleanedticker)))
# cleanedticker.reset_index()
# cleanedticker["__id"] = cleanedticker.index

In [ ]:
with sqlite3.connect('pstocks.sqlite') as con:
    cleanedticker.to_sql('pstocks', con=con, dtype={'__id': 'INTEGER PRIMARY KEY AUTOINCREMENT'})

In [ ]:
# this will push dataframe to sql, creating a table if none exists
# cleanedticker.to_sql('stocks', conn, if_exists='append')

In [ ]:
cleanedticker.head()

In [ ]:
#import csv file with the list of top 100 crypto currencies from yahoo finance
cryptolist = pd.read_csv("yfcryptolist.csv").rename(columns={'Unnamed: 0':'index'})

# print to check df
cryptolist

In [ ]:
# set variable name for list for the for loop
cryptoStrings = cryptolist['Ticker']

In [ ]:
# yfinance code to separate tickers into pretty dataframe
df_list2 = list()
for crypto in cryptoStrings:
    data2 = yf.download(crypto, group_by="Ticker", period='2y')
    # add this column because the dataframe doesn't contain a column with the ticker
    data2['Ticker'] = crypto 
    
    df_list2.append(data2)

# combine all dataframes into a single dataframe
cryptoDf = pd.concat(df_list2)

In [ ]:
# print df
cryptoDf

In [ ]:
# reset index to prepare for merge and rename ticker column
cryptoindex = cryptoDf.reset_index().rename(columns={"Ticker":"Ticker-drop"})

cryptoindex

In [ ]:
# split the hyphen from crypto ticker for the final crypto df, rename columns, and reset index to prepare for merge
tickersplit = cryptolist['Ticker'].str.split('-' ,expand=True).rename(columns={0:'Ticker', 1: 'drop'}).reset_index()
tickersplit


In [ ]:
# split the space from crypto name for the final crypto df, rename columns, and reset index to prepare for merge
namesplit = cryptolist['Name'].str.split(' ', expand = True).rename(columns={0:'Name', 1: 'drop'}).reset_index()
namesplit

In [ ]:
# merge the two dfs
rename_merge = tickersplit.merge(namesplit, how='right', on='index').drop(columns =['drop_x','drop_y',2,3])

rename_merge

In [ ]:
# merge the cryptolist df to prepare for the merge with the crypto data df and rename the columns
mergecrylist = rename_merge.merge(cryptolist, how = 'right', on= 'index')\
            .rename(columns={'Name_x':'Name-drop','Ticker_x':'Ticker','Name_y':'Name','Ticker_y':'Ticker-drop'})

mergecrylist

In [ ]:
# final merge between the data df and the name list df and drop the unnecessary columns
crypto_merge =cryptoindex.merge(mergecrylist, how= 'right', on='Ticker-drop')\
                .drop(columns=['Open','High','Low','Adj Close', 'index', 'Ticker-drop', 'Name-drop'])\
                .set_index('Date')
crypto_merge["Sector"] = "Crypto Currency"

crypto_merge

In [ ]:
#find percent change from day to day
pct_crypto = crypto_merge['Close'].pct_change()*100

#add percent change to df
crypto_merge['ClosePerChange'] = pct_crypto

# dropping Nan rows
dropnan = crypto_merge.dropna()

crypto_cleaned = dropnan

crypto_cleaned

In [ ]:
# export to sql
crypto_cleaned.to_sql('pstocks', conn, if_exists='append')

In [ ]:
sqlBubble = pd.read_sql('SELECT * FROM pstocks', con=conn)
sqlBubble

In [ ]:
sqlBubble["Date"] = pd.to_datetime(sqlBubble['Date']).dt.date
sqlBubble

In [ ]:
with sqlite3.connect('stocks.sqlite') as con:
    sqlBubble.to_sql('stocks', con=con, dtype={'__id': 'INTEGER PRIMARY KEY AUTOINCREMENT'})